<a href="https://colab.research.google.com/github/hg333/Domain-Specific-Lexicons/blob/main/Phone_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle 
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d "PromptCloudHQ/amazon-reviews-unlocked-mobile-phones"
!ls

Saving kaggle.json to kaggle.json
 98% 32.0M/32.6M [00:00<00:00, 49.5MB/s]
100% 32.6M/32.6M [00:00<00:00, 82.4MB/s]
amazon-reviews-unlocked-mobile-phones.zip  kaggle.json	sample_data


In [ ]:
import numpy as np
import re
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('sentiwordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
lemmatizer = WordNetLemmatizer()
!pip install senticnet
from senticnet.senticnet import SenticNet
sn = SenticNet()
import math
import matplotlib.pyplot as plt
import re
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.
     |████████████████████████████████| 51.9MB 68kB/s 


In [ ]:
df= pd.read_csv("amazon-reviews-unlocked-mobile-phones.zip")
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [ ]:
li= list(df['Reviews'])
rating=list(df['Rating'])

In [ ]:
stop_words = set(stopwords.words('english'))
def preprocess(str):
  str=pos_tag(word_tokenize(str))
  ret=[]

  for word,tag in str:
    if word!="br" and word.isalpha() and (word.lower() not in stop_words):
      if tag.startswith('NN'):
        pos = 'n'
      elif tag.startswith('VB'):
        pos = 'v'
      else:
        pos='a'
      ret.append((lemmatizer.lemmatize(word.lower(), pos),tag))
  return ret


In [ ]:


def retpos(tag):
  if tag.startswith('N'):
    return 'n'
  elif tag.startswith('V'):
    return 'v'
  elif tag.startswith('RB'):
    return 'r'
  elif tag.startswith('J'):
    return 'a'

  return None

# def getSWNscore(str):
#     str=str.lower()
#     tagg=pos_tag(word_tokenize(str))
#     ansP=0
#     ansN=0
#     for i in tagg:
#       try:
#         ansP+=swn.senti_synset(i[0]+'.'+retpos(i[1]).lower()+'.03').pos_score()
#         ansN+=swn.senti_synset(i[0]+'.'+retpos(i[1]).lower()+'.03').neg_score()
#       except:
#         continue
#       print(i,ansP,ansN,getscore(i[0]));
    
#     if ansP==0 and ansN==0: return getscore(str)
#     if ansP>=ansN: return ansP
#     else : return -ansN
#     #return (ansP,ansN)


def getMinDistSum(positions):
    def get_norm(point1, point2):
        return math.sqrt((point1[0] - point2[0]) * (point1[0] - point2[0]) + \
                    (point1[1] - point2[1]) * (point1[1] - point2[1]))
    
    # geometric median
    if (len(positions)==0): return((0,0),"NEUTRAL")
    center_x = sum([x for x, y in positions]) / len(positions)
    center_y = sum([y for x, y in positions]) / len(positions)
    center_new = (center_x, center_y)
    center_prev = (center_x - 1.0, center_y - 1.0)
    epox=1
    while epox<50 and get_norm(center_prev, center_new) > 1e-8:
        up_x = 0
        up_y = 0
        down = 0
        for point in positions:
            distance = get_norm(center_new, point)
            if -0.001 < distance < 0.001:
                continue
            up_x += point[0] / distance
            up_y += point[1] / distance
            down += 1 / distance
            
        if down == 0:
            break
            
        center_prev = center_new
        center_new = (up_x / down, up_y / down)
        epox+=1
    res=""
    if center_new[1]>0:
      if center_new[0]<0: res="VERY POS"
      else: res="POS"
    else:
      if center_new[0]<0: res="VERY NEG"
      else: res="NEG"
    return (center_new,res)

In [ ]:
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-200')

[========================================----------] 81.7% 205.9/252.1MB downloaded

In [ ]:
glove_vectors.similarity('phone','bad')

0.287987

In [ ]:

# import these modules 
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
   
ps = PorterStemmer() 
  
# choose some words to be stemmed 
words = ["program", "programs", "programer", "great", "flawless"] 
  
for w in words: 
    print(w, " : ", ps.stem(w)) 

program  :  program
programs  :  program
programer  :  program
great  :  great
flawless  :  flawless


In [ ]:

def getSWNscore2(word,tag):
  if word in stop_words: return 0
  wt=word
  word = ps.stem(word)
  p=0
  n=0
  try:
    #print(wt,retpos(tag))
    for i in swn.senti_synsets(wt,retpos(tag)):
      p=max(p,i.pos_score())
      n=max(n,i.neg_score())
      break
    # p=swn.senti_synset(word+'.'+retpos(tag).lower()+'.01').pos_score()
    # n=swn.senti_synset(word+'.'+retpos(tag).lower()+'.01').neg_score()
  except:
    try:
      for i in swn.senti_synsets(word,retpos(tag)):
        p=max(p,i.pos_score())
        n=max(n,i.neg_score())
        break
    except:
      p=0
      n=0
  #if p==0 and n==0: return getscore(word)
  
  if p>=n: return p
  else : return -n

def getLexiScore(word,tag):
  theta=np.pi*getSWNscore2(word,tag)/2
  r=0
  try:
    r=abs(glove_vectors.similarity('phone',word));
  except:
    r=0

  if (theta==0 or r==0): return False
 # print(word,theta,r);
  return (r*math.cos(theta),r*math.sin(theta))

def evaluate1(str):
  str=str.lower()
  text = str
  str= word_tokenize(str)
  ret=[]
  regex1 = "no|never|nothing|nowhere|noone|none|not|havent|hasnt|hadnt|cant|couldnt|shouldnt|wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint|n't"
  regex1=regex1.split("|")
  #print(regex1)
  for i in regex1:
    if(text.find(i)!=-1) :return False
  str=pos_tag(str);
  sentiment=0
  for word,tag in str:
    if (word!="br" and word.isalpha() and (word.lower() not in stop_words)):
      # if tag.startswith('NN'):
      #   pos = 'n'
      # elif tag.startswith('VB'):
      #   pos = 'v'
      # elif tag.startswith('J'):
      #   pos='a'
      # else : continue
      # #print(word)
      # #t=getLexiScore(lemmatizer.lemmatize(word.lower(), pos))
      t=getLexiScore(word,tag)
      
      if t: 
        ret.append((word,t[1]))
        sentiment+=t[1]
     # print(ret)
  return ret,sentiment # return getMinDistSum(ret)

In [ ]:

def getscore(str):
    ret=0
    try: 
      ret=(float(sn.polarity_value(str)))
    except:
      ret=0
    return (ret)

def getSWNscore(str):
    str=str.lower()
    tagg=pos_tag(word_tokenize(str))
    #print(tagg)
    ansP=0
    ansN=0
    ret=[]
    ans=0
    for i in tagg:
      x=getSWNscore2(i[0],i[1])
      if(x !=0): ret.append((i[0],x))
      ans+=x
    return ret,ans

In [ ]:
pos_tag(word_tokenize("I got it with cover plus Adhesive protection film on both sides"))

[('I', 'PRP'),
 ('got', 'VBD'),
 ('it', 'PRP'),
 ('with', 'IN'),
 ('cover', 'NN'),
 ('plus', 'CC'),
 ('Adhesive', 'JJ'),
 ('protection', 'NN'),
 ('film', 'NN'),
 ('on', 'IN'),
 ('both', 'DT'),
 ('sides', 'NNS')]

In [ ]:
k=evaluate1("I give to you 5 starts because you deserve it. that experience with you was really great I am very happy to have business with you once again thank you and keep doing like that.")
if k:
  print(k)

([('give', -0.07607853071498374), ('deserve', 0.08034800082956256), ('really', 0.2805128603862278), ('happy', 0.2578765056590756), ('like', 0.07685939360093735)], 0.6195182297608195)


In [ ]:
for i in swn.senti_synsets('flawless'):
  print(i)

<flawless.s.01: PosScore=0.0 NegScore=0.375>


In [ ]:
for i in swn.senti_synsets('high'):
  print(i)

<high.n.01: PosScore=0.125 NegScore=0.0>
<high.n.02: PosScore=0.0 NegScore=0.0>
<high.n.03: PosScore=0.0 NegScore=0.125>
<high.n.04: PosScore=0.0 NegScore=0.0>
<high.n.05: PosScore=0.0 NegScore=0.0>
<senior_high_school.n.01: PosScore=0.0 NegScore=0.0>
<high_gear.n.01: PosScore=0.0 NegScore=0.0>
<high.a.01: PosScore=0.125 NegScore=0.25>
<high.a.02: PosScore=0.125 NegScore=0.0>
<eminent.s.01: PosScore=0.375 NegScore=0.125>
<high.a.04: PosScore=0.0 NegScore=0.0>
<high.s.05: PosScore=0.5 NegScore=0.0>
<gamey.s.02: PosScore=0.0 NegScore=0.125>
<high.s.07: PosScore=0.125 NegScore=0.0>
<high.r.01: PosScore=0.0 NegScore=0.0>
<high.r.02: PosScore=0.0 NegScore=0.0>
<high.r.03: PosScore=0.25 NegScore=0.0>
<high.r.04: PosScore=0.0 NegScore=0.0>


In [ ]:
getSWNscore('''At first glance, the phone looked fine, however the photo's I took using my phone looks "foggy". Upon closer inspection, there are gouges/crapes right on the exterior of the camera lens. Also, while this is a 3G phone, it has difficulty connecting. My partner has the exact same phone and when in the house or riding down the highway, his will be on 3G while mine is on 1x... whatever that is. Doing simple processes is delayed as if the processor or RAM is having some problem''')

([('fine', 0.375),
  ('however', -0.5),
  ('foggy', -0.5),
  ('right', 0.25),
  ('difficulty', -0.5),
  ('simple', -0.375),
  ('delayed', 0.125),
  ('problem', -0.625)],
 -1.75)

In [ ]:
li[0]

"I feel so LUCKY to have found this used (phone to us & not used hard at all), phone on line from someone who upgraded and sold this one. My Son liked his old one that finally fell apart after 2.5+ years and didn't want an upgrade!! Thank you Seller, we really appreciate it & your honesty re: said used phone.I recommend this seller very highly & would but from them again!!"

In [ ]:
evaluate1

<function __main__.evaluate1>

In [ ]:
evaluate1('I HATE THIS PHONE, WEEK LATER HAD TO BUY ANOTHER ONE...ITS SLOW AND STORAGE IS VERY POOR...A WASTE OF MONEY...NEVER AGAIN...')

False

In [ ]:
res=[]
interesting=[]
for i in range(len(li)):
  if (li[i]==li[i]):
    k=evaluate1(li[i])
    if (k):
      k2=getSWNscore(li[i])
      res.append([li[i],k[1],k[0],k2[1],k2[0],rating[i]])
      if k2[1]*k[1]<=0:#k[0][1]
        interesting.append([li[i],k,getSWNscore(li[i]),rating[i]])
    if(i%5000==0):
      print(i,end=" ")

0 5000 10000 15000 20000 25000 30000 35000 40000 45000 50000 55000 60000 65000 70000 75000 80000 85000 90000 95000 100000 105000 110000 115000 120000 125000 130000 135000 140000 145000 150000 155000 160000 165000 170000 175000 180000 185000 190000 195000 200000 205000 210000 215000 220000 225000 230000 235000 240000 245000 250000 255000 260000 265000 270000 275000 280000 285000 290000 295000 300000 305000 310000 315000 320000 325000 330000 335000 340000 345000 350000 355000 360000 365000 370000 375000 380000 385000 390000 395000 400000 405000 410000 

In [ ]:
interDF=pd.DataFrame(interesting)

In [ ]:
interDF.head()

,0,1,2,3
0,It's battery life is great. It's very responsi...,"([(issue, 0.05847430649687963), (black, -0.125...","([(issue, 0.125), (black, -0.375), (several, -...",3
1,"as described, fast ship!","([], 0)","([], 0)",5
2,Perfect in every way.,"([], 0)","([], 0)",5
3,This phone came in great condition! Great pric...,"([], 0)","([], 0)",5
4,excellent,"([], 0)","([], 0)",5


In [ ]:
interDF.to_csv('Mismatchs.csv')

In [ ]:
# def evaluate2(str):
#   str= word_tokenize(str)
#   ret=[]
#   if "not" in str or "n't" in str:
#     return False
#   str=pos_tag(str);
#   for word,tag in str:
#     if (word!="br" and word.isalpha() and (word.lower() not in stop_words)):
#       if tag.startswith('NN'):
#         pos = 'n'
#       elif tag.startswith('VB'):
#         pos = 'v'
#       elif tag.startswith('J'):
#         pos='a'
#       else : continue
#       #print(word)
#       t=getLexiScore(lemmatizer.lemmatize(word.lower(), pos))
#       if t: ret.append(t)
#   return ret
# print(evaluate2(li[35]))
# getMinDistSum(evaluate2(li[35]))


In [ ]:
totDF=pd.DataFrame(res)
totDF.to_csv('res.csv')

In [ ]:
filtered=interDF[interDF[1] != ([],0)]

In [ ]:
filtered.head()

,0,1,2,3
0,It's battery life is great. It's very responsi...,"([(issue, 0.05847430649687963), (black, -0.125...","([(issue, 0.125), (black, -0.375), (several, -...",3
20,"Phone works great, you may have to get a sum c...","([(get, 0.08728428872216379), (micro, -0.06376...","([(get, 0.125), (micro, -0.25)], -0.125)",4
22,my autistic son is in his glory lol thank you ...,"([(autistic, -0.012012857140949955), (glory, 0...","([(autistic, -0.75), (glory, 0.25)], -0.5)",5
25,More complicated than I expected. Terrible rec...,"([(complicated, -0.21476260947696374), (expect...","([(complicated, -0.625), (expected, 0.25), (te...",1
26,phone reception poor with incoming calls,"([(reception, 0.1964319623933893), (poor, -0.1...","([(reception, 0.5), (poor, -0.875)], -0.375)",1


In [ ]:
templi=list(filtered.values)

In [ ]:
len(templi)

6070

In [ ]:
templi[0][1]

([('issue', 0.05847430649687963),
  ('black', -0.12545547852196717),
  ('several', -0.261077494062399),
  ('times', 0.25004496640879337),
  ('get', 0.08728428872216379)],
 0.009270589043470584)

In [ ]:
success=[]
failure=[]

In [ ]:
for i in range(len(templi)):
  #print(i,templi[i],templi[i][1][0],templi[i][3]-3)
  if templi[i][1][1]*(templi[i][3]-3)>0:
    success.append(templi[i])
  elif templi[i][1][1]*(templi[i][3]-3)<0:
    failure.append(templi[i])

In [ ]:
len(success)

2290

In [ ]:
len(failure)

3187

In [ ]:
sucDF=pd.DataFrame(success)

In [ ]:
sucDF.to_csv('success.csv')

In [ ]:
negDF=pd.DataFrame(failure)
negDF.to_csv('failures.csv')

In [ ]:
negDF.head

<bound method NDFrame.head of                                                       0  ...  3
0     More complicated than I expected. Terrible rec...  ...  1
1              phone reception poor with incoming calls  ...  1
2                                Good basic smart phone  ...  5
3     IF IS A SMARTPHONE EXCELLENT dig a little, IN ...  ...  4
4     Good Performance, Touchscreen and Display OKGo...  ...  5
...                                                 ...  ... ..
3182  i love those phones.I bought two of them as a ...  ...  5
3183  Have had several of these phones over the year...  ...  5
3184  My son enjoys the flip phone...it works great ...  ...  4
3185  Bought this phone for my husband to replace an...  ...  5
3186  This is a great little phone - very sturdy. My...  ...  5

[3187 rows x 4 columns]>

In [ ]:
evaluate1("all very well and fast, and all this and that is described in the publication recommended for future purchases recommended 100%")

([('well', 0.17937370829383192),
  ('recommended', -0.02997221230267179),
  ('recommended', -0.02997221230267179)],
 0.11942928368848836)

In [ ]:
getSWNscore("all very well and fast, and all this and that is described in the publication recommended for future purchases recommended 100%")

([('well', 0.375),
  ('is', 0.25),
  ('recommended', -0.125),
  ('recommended', -0.125)],
 0.375)

In [ ]:
getSWNscore('like')

('like', 'IN') 0.0 0.25


-0.25

In [ ]:
glove_vectors.similarity('phone','cheap')

0.3304398

#ROUGH WORK

In [ ]:
def getSWNscore2(word,tag):
  print(word,tag)
  try:
    p=swn.senti_synset(word+'.'+retpos(tag).lower()+'.01').pos_score()
    n=swn.senti_synset(word+'.'+retpos(tag).lower()+'.01').neg_score()
  except:
    p=0
    n=0
  if p==0 and n==0: return getscore(word)
  
  if p>=n: return p
  else : return -n

def getLexiScore(word,tag):
  theta=np.pi*getSWNscore2(word,tag)/2
  r=0
  try:
    r=abs(glove_vectors.similarity('phone',word));
  except:
    r=0

  if (theta==0 or r==0): return False
 # print(word,theta,r);
  return (r*math.cos(theta),r*math.sin(theta))

def evaluate1(str):
  str=str.lower()
  text = str
  str= word_tokenize(str)
  print(str)
  ret=[]
  regex1 = "no|never|nothing|nowhere|noone|none|not|havent|hasnt|hadnt|cant|couldnt|shouldnt|wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint|n't"
  regex1=regex1.split("|")
  #print(regex1)
  for i in regex1:
    if(text.find(i)!=-1) :return False
  str=pos_tag(str);
  print(str)
  sentiment=0
  for word,tag in str:
    if (word!="br" and word.isalpha() and (word.lower() not in stop_words)):
      if tag.startswith('NN'):
        pos = 'n'
      elif tag.startswith('VB'):
        pos = 'v'
      elif tag.startswith('J'):
        pos='a'
      else : continue
      #print(word)
      #t=getLexiScore(lemmatizer.lemmatize(word.lower(), pos))
      t=getLexiScore(word,tag)
      
      if t: 
        ret.append(t)
        sentiment+=t[1]
      print(word,t)
  return sentiment # return getMinDistSum(ret)
def getscore(str):
    ret=0
    try: 
      ret=float(sn.polarity_value(str))
    except:
      ret=0
    return (ret)

def getSWNscore(str):
    str=str.lower()
    tagg=pos_tag(word_tokenize(str))
    #print(tagg)
    ansP=0
    ansN=0
    for i in tagg:
      p=0
      n=0
      try:
        p+=swn.senti_synset(i[0]+'.'+retpos(i[1]).lower()+'.01').pos_score()
        n+=swn.senti_synset(i[0]+'.'+retpos(i[1]).lower()+'.01').neg_score()
      except:
        p=0
      if p==0 and n==0:
        scr=getscore(i[0]);
        #print(scr)
        if scr>0: p+=scr
        else: n-=scr
      if(p+n!=0):
        print(i[0],p-n)
      ansP+=p
      ansN+=n 
    if ansP==0 and ansN==0: return getscore(str)
      #ansP+=getscore(i[0])
      #print(i[0],getscore(i[0]))
    #return ansP
    if ansP>=ansN: return ansP
    else : return -ansN
    #return (ansP,ansN)

In [ ]:
evaluate1("""It came quickly and the phone looks like its brand new. My only issue is that it does some weird things whenI try to use it while its charging. Other than that it works perfectly!""")

['it', 'came', 'quickly', 'and', 'the', 'phone', 'looks', 'like', 'its', 'brand', 'new', '.', 'my', 'only', 'issue', 'is', 'that', 'it', 'does', 'some', 'weird', 'things', 'wheni', 'try', 'to', 'use', 'it', 'while', 'its', 'charging', '.', 'other', 'than', 'that', 'it', 'works', 'perfectly', '!']
[('it', 'PRP'), ('came', 'VBD'), ('quickly', 'RB'), ('and', 'CC'), ('the', 'DT'), ('phone', 'NN'), ('looks', 'VBZ'), ('like', 'IN'), ('its', 'PRP$'), ('brand', 'NN'), ('new', 'JJ'), ('.', '.'), ('my', 'PRP$'), ('only', 'JJ'), ('issue', 'NN'), ('is', 'VBZ'), ('that', 'IN'), ('it', 'PRP'), ('does', 'VBZ'), ('some', 'DT'), ('weird', 'JJ'), ('things', 'NNS'), ('wheni', 'VBP'), ('try', 'NN'), ('to', 'TO'), ('use', 'VB'), ('it', 'PRP'), ('while', 'IN'), ('its', 'PRP$'), ('charging', 'NN'), ('.', '.'), ('other', 'JJ'), ('than', 'IN'), ('that', 'IN'), ('it', 'PRP'), ('works', 'VBZ'), ('perfectly', 'RB'), ('!', '.')]
came VBD
came False
phone NN
phone False
looks VBZ
looks False
brand NN
brand False
ne

0.018323785193458092

In [ ]:
getSWNscore("""It came quickly and the phone looks like its brand new. My only issue is that it does some weird things whenI try to use it while its charging. Other than that it works perfectly!.""")

quickly -0.64
like -0.25
new 0.375
issue 0.125
weird -0.75
charging -0.64
other -0.625
works 0.9
perfectly 0.5


-2.9050000000000002

In [ ]:
getSWNscore("100% satisfied")

[('100', 'CD'), ('%', 'NN'), ('satisfied', 'VBD')]


0

In [ ]:
evaluate1('This phone is terrible. International model runs on 2g (T-Mobile) and the screen went dead a week after receiving it.')

lexi phone 0.0 1.0
lexi terrible -0.9817477042468103 0.5982422530651093
lexi terrible -0.9817477042468103 0.5982422530651093
lexi international -0.7853981633974483 0.6078873500227928
lexi international -0.7853981633974483 0.6078873500227928
lexi model 1.2849113953182254 0.6080478429794312
lexi model 1.2849113953182254 0.6080478429794312
lexi runs 0.0 0.6225761994719505
lexi screen 0.0 0.6904341876506805
lexi went 0.0 0.6883260756731033
lexi dead -1.1780972450961724 0.6394045650959015
lexi dead -1.1780972450961724 0.6394045650959015
lexi week 0.0 0.7071676552295685
lexi receiving 1.413716694115407 0.704303503036499
lexi receiving 1.413716694115407 0.704303503036499
[(0.3323655879375546, -0.49742025421904756), (0.4298412673986372, -0.42984126739863715), (0.17147349018208893, 0.583368684038415), (0.2446895336408071, -0.5907327906863841), (0.11017734172347537, 0.6956323581894623)]


((0.34215492952043924, -0.42056495556050416), 'NEG')

In [ ]:
getSWNscore('dead')

0.0

In [ ]:
np.pi*0.875

2.748893571891069

In [ ]:
np.pi/2

1.5707963267948966

In [ ]:
glove_vectors.similarity('phone','perfectly')

0.10353342

In [ ]:

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
 
 
lemmatizer = WordNetLemmatizer()
 
 
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None
 
 
def clean_text(text):
    text = text.replace("<br />", " ")
    #text = text.decode("utf-8")
 
    return text
 
 
def swn_polarity(text):
    """
    Return a sentiment polarity: 0 = negative, 1 = positive
    """
 
    sentiment = 0.0
    tokens_count = 0
 
    text = clean_text(text)
 
 
    raw_sentences = sent_tokenize(text)
    print(raw_sentences)
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        print(tagged_sentence)
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
 
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
 
    # judgment call ? Default to positive or negative
    return sentiment

In [ ]:
swn_polarityliked it.')

['amazing,liked it.']
[('amazing', 'NN'), (',', ','), ('liked', 'VBD'), ('it', 'PRP'), ('.', '.')]


0.0

In [ ]:
preprocess('Screen freezes a lot but other than that i like it')

[('screen', 'NNP'), ('freeze', 'VBZ'), ('lot', 'NN'), ('like', 'IN')]

In [ ]:
getSWNscore2('like','IN')

-0.25

In [ ]:
getSWNscore('It was scratched')

('it', 'PRP') 0 0
('was', 'VBD') 0 0
('scratched', 'VBN') 0 0


0

In [ ]:
swn.senti_synset('scrath'+'.'+retpos('NNS')+'.03')

WordNetError: ignored

In [ ]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
lemmatizer.lemmatize('scratches')

'scratch'

#rough work 2


In [ ]:

# import these modules 
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
   
ps = PorterStemmer() 
  
# choose some words to be stemmed 
words = ["program", "programs", "programer", "great", "excellent"] 
  
for w in words: 
    print(w, " : ", ps.stem(w)) 

program  :  program
programs  :  program
programer  :  program
great  :  great
excellent  :  excel


In [ ]:
k=wn.synsets('like','n')[0]

In [ ]:
for i in swn.senti_synsets('like','n')

SyntaxError: ignored

In [ ]:
h=swn.senti_synsets('like','v')

In [ ]:
for i in h:
  print(i,i.pos_score())

<wish.v.02: PosScore=0.125 NegScore=0.0> 0.125
<like.v.02: PosScore=1.0 NegScore=0.0> 1.0
<like.v.03: PosScore=0.375 NegScore=0.0> 0.375
<like.v.04: PosScore=0.375 NegScore=0.0> 0.375
<like.v.05: PosScore=0.125 NegScore=0.0> 0.125


In [ ]:
from nltk.corpus.reader import CorpusReader

In [ ]:
k=SentiWordnetCorpusReader('This is great')

NameError: ignored